# OAIS Numerics

In [20]:
using LinearAlgebra, Distributions, AutoGrad

In [21]:
function sample_gaussian(μ, Σ)
    return rand(MvNormal(μ, Σ))
end

sample_gaussian (generic function with 1 method)

Say we have a normal distribution with mean $\mu^t=\begin{pmatrix} 1\\ 1\end{pmatrix}$ and standard deviation $\Sigma^t = \begin{pmatrix} 5 & 3\\ 3 & 5 \end{pmatrix}$.

In [10]:
μᵗ = [1;1]
Σᵗ = [5 3; 3 5]

function πᶠ(x)
    ret = 1/(2*π*sqrt(det(Σᵗ)))*exp.(-0.5.*(x-μᵗ)'*inv(Σᵗ)*(x-μᵗ))
    ret
end



πᶠ (generic function with 1 method)

We now initialise our proposal to be a standard 2D normal distribution with mean $\mu_0 = \begin{pmatrix} 0\\ 0\end{pmatrix}$ and standard deviation $\Sigma_0 = \begin{pmatrix} 1 & 0\\ 0 & 1 \end{pmatrix}$. 

Recall we are trying to optimise the rho function, which is the ratio of the target distribution to the proposal distribution. 

$$
\rho(\theta) = \mathbb{E}\left[\frac{}{}\right]
$$

In [12]:
πᶠ([1;3])

1-element Vector{Float64}:
 0.02129737554880663

In [80]:
function q(θₖ, x)
    μₖ, Σₖ = θₖ
    1/(2*pi*sqrt(det(Σₖ)))*exp(-0.5*(x-μₖ)'*inv(Σₖ)*(x-μₖ))
end

function ρ(θₖ, N=1_000)
    μₖ, Σₖ = θₖ
    x = sample_gaussian(μₖ, Σₖ)
    ret = (πᶠ(x).^2)./(q(θₖ, x).^2)/N
    ret
end

function ∇ρ(θₖ, N=1_000, h=1e-6)
    μₖ, Σₖ = θₖ
    ∂ρ∂μ = (ρ([μₖ.+h, Σₖ], N) - ρ([μₖ.-h, Σₖ], N))/(2h) .* ones(size(μₖ))
    ∂ρ∂Σ = (ρ([μₖ, Σₖ.+h], N) - ρ([μₖ, Σₖ.-h], N))/(2h) .* ones(size(Σₖ))
    [∂ρ∂μ, ∂ρ∂Σ]
end

function mc(φ, θₖ, N=1_000)
    # montecarlo integral
    μₖ, Σₖ = θₖ
    x = [sample_gaussian(μₖ, Σₖ) for i in 1:N]
    q_t = x -> q(θₖ, x)
    wₜ = πᶠ.(x)./(q_t.(x))
    ret = sum(wₜ .* φ.(x))/N
    ret
end

f = x -> √(x[1]^2 + x[2]^2)


mc (generic function with 2 methods)

Optimization loop

In [85]:
lr = 0.01
θₖ = [[1;1], [5 3; 3 5]]
mcs = []
for i in 1:100
    θₖ = θₖ - lr .* ∇ρ(θₖ)
    eigen(θₖ[1])[1][1] < 0
    push!(mcs, mc(f, θₖ))
end

PosDefException: PosDefException: matrix is not positive definite; Cholesky factorization failed.